In [2]:
#%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
  
print(tf.version)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


# Helper libraries
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn.utils import compute_class_weight

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from pathlib import Path
import os.path
#import datetime, os

<module 'tensorflow._api.v2.version' from 'c:\\Users\\elec_\\anacon\\anaconda3\\envs\\MLIS2car\\lib\\site-packages\\tensorflow\\_api\\v2\\version\\__init__.py'>
Num GPUs Available:  0


In [3]:
batch_size = 16
img_size = [240, 320]

df = pd.read_csv(r'machine-learning-in-science-ii-2023/training_norm.csv')
df['filename'] = df["image_id"].astype(str) + ".png"
#df['label'] = [df["angle"], df["speed"]]
print(df)


x_train = df[0:int(len(df) * 0.8)]
x_validate = df[int(len(df) * 0.8):]

training_images = tf.keras.preprocessing.image_dataset_from_directory(
    "machine-learning-in-science-ii-2023/training_data",
    image_size=img_size,
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=1,
)

# testing = tf.keras.preprocessing.image_dataset_from_directory(
#     "machine-learning-in-science-ii-2023/testing_data/testing_data",
#     image_size=img_size,
#     batch_size=batch_size,
# )


# datagen = ImageDataGenerator(
#     zoom_range=0.2,
#     height_shift_range=0.2,
#     width_shift_range=0.2,
#     shear_range=0.2,
#     fill_mode='nearest')




       image_id   angle  speed   filename
0             1  0.4375    0.0      1.png
1             2  0.8125    1.0      2.png
2             3  0.4375    1.0      3.png
3             4  0.6250    1.0      4.png
4             5  0.5000    0.0      5.png
...         ...     ...    ...        ...
13788     13794  0.6250    1.0  13794.png
13789     13795  0.4375    1.0  13795.png
13790     13796  0.5625    0.0  13796.png
13791     13797  0.6250    0.0  13797.png
13792     13798  0.6875    1.0  13798.png

[13793 rows x 4 columns]
Found 13798 files belonging to 1 classes.
Using 11039 files for training.


In [4]:
# plt.figure(figsize=(12, 10))
# for images, labels in training_images.take(1):
#     for i in range(16):
#         ax = plt.subplot(4, 4, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         #plt.title(training.class_names[labels[i]])
#         plt.axis("off")

In [5]:
training_datagen = ImageDataGenerator(
    rescale = 1./255,
    fill_mode='nearest')

train_generator = training_datagen.flow_from_dataframe(
    dataframe=x_train,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["angle","speed"],
    target_size=img_size,
    batch_size=16,
    class_mode='other')

validation_datagen = ImageDataGenerator(rescale = 1./255)

val_generator = validation_datagen.flow_from_dataframe(
    dataframe=x_validate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["angle","speed"],
    target_size=img_size,
    class_mode='other')

Found 11034 validated image filenames.
Found 2759 validated image filenames.


In [6]:
shape = (*img_size, 3) # inherited image size with 3 color filters

# CNN base
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=shape))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(16, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
        
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Dropout(0.2))
model.add(layers.Flatten()),

model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.7))

model.add(layers.Dense(64, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.2))

model.add(layers.Dense(32, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.1))

# Output layer
model.add(layers.Flatten())
model.add(layers.Dense(2,  activation='relu', kernel_initializer='normal'))


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 238, 318, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 238, 318, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 119, 159, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 117, 157, 16)      4624      
                                                                 
 batch_normalization_1 (Batc  (None, 117, 157, 16)     64        
 hNormalization)                                                 
                                                        

In [8]:
model.compile(
    optimizer='adam',
    loss='mean_squared_error'
)

history = model.fit(
    train_generator, 
    batch_size=batch_size,
    validation_data=val_generator,
    epochs=1
)

690/690 [==============================] - 349s 500ms/step - loss: 0.1259 - val_loss: 0.0506


In [9]:
#model = tf.keras.models.load_model('model.h6')

test_images = tf.keras.preprocessing.image_dataset_from_directory(
    "machine-learning-in-science-ii-2023/test_data",
    image_size=img_size,
    seed=1,
)

normalization_layer = tf.keras.layers.Rescaling(1./255)

test_images = test_images.map(lambda x, y: (normalization_layer(x), y))


#I have no idea if this will do the right scaling and stuff- > fixing that might fix everyhing who knows.

#training_datagen = ImageDataGenerator(
#    rescale = 1./255,
#    fill_mode='nearest')

#train_generator = training_datagen.flow_from_dataframe(
#    dataframe=x_train,
#    directory="machine-learning-in-science-ii-2023/test_data",
#    x_col="filename",
#    y_col=["angle","speed"],
#    target_size=img_size,
#    class_mode='other')


prediction = model.predict(test_images)
print(prediction.shape)

#indexes=list(range(len(prediction)))
#iad=[x+1 for x in indexes]
a= prediction[:,0]
b= prediction[:,1]


submissiondata={"angle":a , "speed":b} 
df = pd.DataFrame(submissiondata)
df.index += 1 
df.to_csv('Submission.csv')


Found 1020 files belonging to 1 classes.
32/32 [==============================] - 8s 229ms/step
(1020, 2)


In [10]:
from sklearn.metrics import classification_report

model = tf.keras.models.load_model('secondModel.h5')

test_loss, test_acc = model.evaluate(
    testing,
    verbose=1
)


predict_model = model.predict(testing, verbose =1)

y = np.concatenate([y for x, y in testing], axis=0)
true = np.argmax(y, axis = 1)

print(true)
predict_model = np.argmax(predict_model, axis = 1)
print(predict_model)

print(classification_report(true, predict_model, target_names=class_names))


OSError: No file or directory found at secondModel.h5